<a href="https://colab.research.google.com/github/SrinithiSL/Yolov1_implementation/blob/main/Yolov1_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [2]:
architecture_config=[(7,64,2,3),'M',(3,192,1,1),'M',(1,128,1,1),(3,256,1,0),(1,512,1,1),(3,512,1,0),'M',[(1,256,1,0),(3,512,1,1),4],(1,512,1,0),(3,1024,1,1),'M',[(1,512,1,0),(3,1024,1,1),2],(3,1024,1,1),(3,1024,2,1),(3,1024,1,1),(3,1024,1,1)]

In [3]:
class block(nn.Module):
  def __init__(self,input_channels,output_channels,**kwargs):
    super(block, self).__init__()
    self.conv = nn.Conv2d(input_channels, output_channels,  bias=False,**kwargs)
    self.bn = nn.BatchNorm2d(output_channels)
    self.leakyrelu=nn.LeakyReLU(0.1)

  def forward(self,x):
    return self.leakyrelu(self.bn(self.conv(x)))  


In [4]:
class yolov1(nn.Module):
  def __init__(self,input_channels=3,**kwargs):
    super(yolov1, self).__init__()
    self.architecture=architecture_config
    self.input_channels=input_channels
    self.convnet=self.create_layer(self.architecture)
    self.fcs=self.create_fcs(**kwargs)

  def forward(self,x):
    x=self.convnet(x)
    return self.fcs(torch.flatten(x,start_dim=1))

  def create_layer(self,architecture):  
    layers=[]
    input_channels=self.input_channels

    for x in architecture:
      if type(x)==tuple:
        layers+=[block(input_channels,output_channels=x[1],kernel_size=x[0],stride=x[2],padding=x[3],)]
        input_channels=x[1]
      elif type(x)==str:
        layers+=[nn.MaxPool2d(kernel_size=2,stride=2)]
      elif type(x)==list:
        conv1=x[0]
        conv2=x[1]
        num_repeats=x[2]

        for _ in range(num_repeats):
          layers+=[block(input_channels,conv1[1],kernel_size=conv1[0],stride=conv1[2],padding=conv1[3])]  
          layers+=[block(conv1[1],conv2[1],kernel_size=conv2[0],stride=conv2[2],padding=conv2[3])]  
          input_channels=conv2[1]

    return nn.Sequential(*layers)      

  def create_fcs(self,split_size,num_boxes,num_classes):
    s,b,c=split_size,num_boxes,num_classes
    return nn.Sequential(nn.Flatten(),nn.Linear(1024*s*s,4096),nn.Dropout(0.0),nn.LeakyReLU(0.1),nn.Linear(4096,s*s*(c+b*5)))


In [5]:
def test(s=7,b=2,c=20):
  model=yolov1(split_size=s,num_boxes=b,num_classes=c)
  x=torch.randn((2,3,448,448))
  print(model(x).shape)

In [6]:
test()

torch.Size([2, 1470])
